# Introdução

A Prefeitura de Fortaleza-CE divulga, por meio da Secretaria Municipal da Saúde, as listas de agendados, cadastrados e vacinados no portal de informações da prefeitura.

Este script baixa os arquivos PDF que contém a [listas de agendados](https://coronavirus.fortaleza.ce.gov.br/vacinacao.html) da vacina contra o COVID19 em Fortaleza/CE, procura pelo nome dado e envia o resultado da busca para uma conta push bullet através da sua API

Autor: Antonio Alisio de Meneses Cordeiro - alisio.meneses@gmail.com

![Screenshot](screenshot-coronaportal.png)

# Pré-requisitos

## Linux
* Conta [pushbullet](https://www.pushbullet.com)
* [Token de acesso](https://docs.pushbullet.com) do pushbullet
* Computador linux com os seguinte pacotes instalados:
  * Python3
  * pdfgrep
  * wget

## MacOS
* Conta [pushbullet](https://www.pushbullet.com)
* Token de acesso do [pushbullet](https://docs.pushbullet.com)
* Gerenciador de pacotes [homebrew](https://brew.sh)
* Computador Mac com os seguinte pacotes instalados:
  * Python3
  * pdfgrep
  * wget

# Instalação

## Linux

Instalar dependencias (pacotes e bibliotecas) no linux debian like:
```sh
pip install -r requirements.txt
apt install wget pdfgrep
```
## MacOS

Instalar dependencias (pacotes e bibliotecas) no MacOS:
```sh
pip3 install -r requirements.txt
brew install pdfgrep
```

# Configuração

Para obter o token de acesso do pushbullet basta:

1. Acessar e logar no pushbullet pelo browser
1. Clicar na seção 'Settings > Account'
1. Clicar no botão 'Create Access Token'
1. Copiar o token gerado

Conforme ilustrado na animação a seguir :

![](pushbullet_access_token.gif)

Obs. : O token utilizado na animação não é válido.

# Uso:

```sh
./agendacovid19.py -n '<NOME_COMPLETO>' -t '<PUSHBULLET_TOKEN>'
```

Caso exista agendamento para o nome dado, o script enviará as informações para o pushbullet e para a saída padrão, conforme o exemplo a seguir:

```sh
./agendacovid19.py -n 'MARIA DO SOCORRO FULANA SICRANA' -t 'asdkjbadbakjbdbkad '
Mensagem enviada para pushbullet
Encontrado agendamento para MARIA DO SOCORRO FULANA SICRANA: ./arquivos_baixados/03.04Lista_Agendados_03.04.pdf:63.0      MARIA DO SOCORRO FULANA SICRANA                ALDEOTA                UAPS DR. ROBERTO DA SILVA BRUNO        2021-04-03   10:00:00   1   1956-11-19
```

# Limitações:

Testado no Ubuntu 18.04 e no MacOS BigSur

# Melhorias:

Todas. Este script é extremamente simples e tem como alvo não programadores ou
curiosos querendo aprender sobre linux, python e web API.

# Referencias:

* https://coronavirus.fortaleza.ce.gov.br/listaVacinacao.html
* https://docs.pushbullet.com

# Licença

Checar arquivo LICENSE

In [34]:
!pip3 install beautifulsoup4==4.9.3 Flask==1.1.2 lxml==4.6.3 pushbullet.py==0.12.0 requests==2.25.1


In [35]:
# importar bibliotecas
from bs4 import BeautifulSoup
from flask import Flask
from pushbullet import Pushbullet

import requests
import os
import getopt,sys
import subprocess

Definir constantes

In [50]:
# A URL a seguir contém um XML gerado dinamicamente listando os
# arquivos PDF com os agendamentos
url = 'https://spreadsheets.google.com/feeds/list/1IJBDu8dRGLkBgX72sRWKY6R9GfefsaDCXBd3Dz9PZNs/14/public/values'
pushbullet_token='<TOKEN_PUSHBULLET>'
nome = 'FULANO SICLANO DA SILVA'

In [51]:
## Variáveis recebidas pela linha de comando
full_cmd_arguments = sys.argv
argument_list = full_cmd_arguments[1:]
short_options = "t:n:"
long_options = ["token", "nome="]

In [52]:
try:
    pasta_de_download
except:
    pasta_de_download = './arquivos_baixados'

Criar lista contendo todos os endereços dos arquivos PDF para download listados no endereço da variável url

In [53]:
def scrape_lista_de_pdf(url):
    pdf_doc = requests.get(url).text
    soup = BeautifulSoup(pdf_doc, 'xml')
    lista_de_pdf = [ item.pdf.text for item in soup.find_all('entry') ]
    return lista_de_pdf

Criar pasta de download

In [54]:
def criar_pasta_de_download(pasta_de_download):
    try:
        if not os.path.isdir(pasta_de_download):
            os.mkdir(pasta_de_download)
    except OSError:
        print ("Criação do diretório de download falhou %s " % pasta_de_download)

Baixar os arquivos somente se já não existirem localmente

In [55]:
def download_arquivo(url,pasta_de_download='.'):
    nome_do_arquivo = url.rsplit('/', 1)[-1]
    caminho = os.path.join(pasta_de_download, nome_do_arquivo)
    try:
        if not os.path.isfile(caminho):
            arquivo_stream = requests.get(url, stream=True)
            with open(caminho, 'wb') as local_file:
                for data in arquivo_stream:
                    local_file.write(data)
    except:
        print('Não foi possível salvar o arquivo na url {}'.format(url))

A função a seguir executa o aplicativo 'pdfgrep' instalado no sistema p/buscar o nome nos arquivos PDF

In [56]:
def procura_nome_pdfgrep(nome, pasta='.'):
    resultado_da_busca = subprocess.getoutput('pdfgrep -i "{}" {}/*.pdf 2> /dev/null'.format(nome, pasta))
    return resultado_da_busca

In [57]:
def ajuda():
    print("uso: {} -n <NOME_A_SER_BUSCADO> -t <TOKEN_PUSHBULLET>".format(sys.argv[0]))

In [58]:
## Inicio

In [59]:
criar_pasta_de_download(pasta_de_download)

In [60]:
lista_de_links=scrape_lista_de_pdf(url)
for link in lista_de_links:
    download_arquivo(link, pasta_de_download)

Popular as variáveis com as opções passadas

In [ ]:
resultado = procura_nome_pdfgrep(nome, pasta_de_download)
pb = Pushbullet(pushbullet_token)

if resultado != "":
    titulo = "Encontrado agendamento para {}".format(nome)
    push = pb.push_note(titulo, resultado)
    print('Mensagem enviada para pushbullet')
    print("Encontrado agendamento para {}: {}".format(nome,resultado))
else:
    print ("Não foi encontrado agendamento para {}".format(nome))
